In [129]:
import pandas as pd
import numpy as np
from numba import njit, jit
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [24]:
# if last val of chunk is smaller than first val of chunk, its a new chromosome w new starts
# max of array is index of new chromomsome
# NOT A GOOD FUNCTION EVEN WITH NJIT BC TIME TO READ CHUNK OUTWEIGHS MOST
@njit
def new_chr(arr):
    if arr[-1] < arr[0]:
        inv_arr = arr[::-1]
        # max of the chunk will be index of the end of the chromosome
        return len(inv_arr) - np.argmax(inv_arr) - 1

In [ ]:
chunksize=10**6
for index, chunk in enumerate(pd.read_csv("databases/hg38_exac03.txt", sep='\t', usecols=[1], chunksize=chunksize)):#, dtype={"Start":"int"})):
    # index of all the starts per chunk
    idx_arr = chunk["Start"].to_numpy()
    # if last val of chunk is smaller than first val of chunk, its a new chromosome w new starts
    result = new_chr(idx_arr)
    if result != None:
        print(result + index * chunksize)
        break
        
print('Change chunksize for smaller files, remove ["Start"] for undefined columns')
#     if idx_arr[-1] < idx_arr[0]:
#         max of the chunk will be index of the end of the chromosome
#         print(np.amax(idx_arr))

In [89]:
def sep_int(raw_int: int) -> str:
    return f'{raw_int:_}'

In [147]:
def exacfunc(prev_idx: int) -> int:
    chunksize=10**5
    # index + 1 so it begins at new chromosome
    for index, chunk in enumerate(pd.read_csv("databases/hg38_mcap.txt", squeeze=True, sep='\t', usecols=[1], skiprows=prev_idx, chunksize=chunksize)):#, dtype={"Start":"int"})):
        # index of all the starts per chunk
        idx_arr = chunk.to_numpy()
        # if last val of chunk is smaller than first val of chunk, its a new chromosome w new starts
        result = new_chr(idx_arr)
        if result != None:
            # +1 because of indexing in pandas
            return sep_int(result + prev_idx + index * chunksize + 1)

    return 'Change chunksize for smaller files, remove ["Start"] for undefined columns'
#     if idx_arr[-1] < idx_arr[0]:
#         max of the chunk will be index of the end of the chromosome
#         print(np.amax(idx_arr))

In [143]:
def exacfunc_all(num_chr: int) -> list:
    total_list = []
    curr_num = 1
    for x in range(num_chr):
        exac_output = exacfunc(curr_num)
        total_list.append("Chr%s: %s" % (x+1, exac_output))
        curr_num = int(exac_output) + 1
    return total_list

In [152]:
output = exacfunc_all(20)
for x in output:
    print(x)

In [156]:
exacfunc(27_518_378 + 1)

'30_951_587'